# Final Assignment Part 2

## Scenario
### Objective
The objective of this part of the Final Assignment is to analyze the historical trends in automobile sales during recession periods, as you did in the previous part. The goal is to provide insights into how the sales of XYZAutomotives, a company specializing in automotive sales, were affected during times of recession.

In this final assignment, you will have the opportunity to demonstrate the **Dashboarding skills** you have acquired in this course.

### Aims
This lab aims to assess your abilities in creating various visualizations using <u>Plotly and Dash</u>. As a data scientist, you have been given a task to prepare a report on your findings from Automobile Sales data analysis.
You decided to develop a dashboard representing two main reports:
* Yearly Automobile Sales Statistics
* 
Recession Period Statistic

*NOTE*: Year range is between 1980 and 2013.s


## Components of Report Items
<b>1. Yearly Automobile Sales Statistics </b>

This report mainly consists of the following items


Yearly Automobile Sales Using Line Chart for the Whole Period (line cha rt):This line chart displays the average automobile sales for each year across the entire per
d.

Total Monthly Automobile Sales Using Line Chart (line c hart):This line chart displays the total monthly automobile sales for the selected
ear.

Average Vehicles Sold by Vehicle Type in the Selected Year (bar  chart):This bar chart shows the average number of vehicles sold for each vehicle type in the select
 year.

Total Advertisement Expenditure for Each Vehicle Using Pie Chart (p e chart):
This pie chart represents the total advertising expenditure for each vehicle  ype in the selec<b>ted year.

2. Recession Peri</b>od Statistics

Average Automobile Sales Fluctuation Over Recession Period (Year-wise):This line chart shows the average automobile sales for each year during recession periods.

Average Number of Vehicles Sold by Vehicle Type:This bar chart displays the average number of vehicles sold for each vehicle type during recession periods.

Total Expenditure Share by Vehicle Type During Recessions:This pie chart represents the total advertising expenditure share by vehicle type during recession periods.

Effect of Unemployment Rate on Vehicle Type and Sales:This bar chart shows the effect of unemployment rate on automobile sales by vehicle type during recession periods.

NOTE: You have worked creating a dashboard components in Flight Delay Time Statistics Dashboard section. You will be working on the similar lines for this Dashboard

In [17]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [107]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt
#Create app
app = dash.Dash(__name__)
#Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True
# Read the Final Assignment Dataset data into pandas dataframe
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df = pd.read_csv(URL)
#Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name() #used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year
year_list = [i for i in range(1980, 2024)]
#Layout Section of Dash
#Task 1 Add the Title to the Dashboard
app.layout = html.Div(children=[html.H1('Automobile Sales Stastistics Dashboard', 
                                style={'textAlign': 'left', 'color': '#503D36',
                                'font-size': 24}),
                              dcc.Dropdown(id='dropdown-statistics',
                                          options=[
                                              'Yearly Statistics','Recession Period Statistics'
                                          ],
                                          value='Select Statistics',
                                          placeholder = 'Select a report type'),
                               dcc.Dropdown(id='select-year',
                                           options=year_list,
                                           value='select-year',
                                           placeholder='select-year'),
                               html.Div([html.Div(
                                   id='output-container', 
                                   className='chart-grid', 
                                   style={'display': 'flex'})])
                               ])

# Define the callback function
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)

def update_input_container(stat):
    if stat == 'Yearly Statistics':
        return False
    else:
        return True

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'), Input(component_id='select-year', component_property='value')]
)

def update_output_container(stat, year):
    if stat == 'Recession Period Statistics':
        recession_data = df[df['Recession'] == 1]
         # grouping data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph (Plot 1)
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile sales fluctuation over Recession Period (year wise)"))
        
        #Plot 2: Bar Chart
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                 
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="Average number of vehicles sold by vehicle type"))


        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        # Hint:Use Vehicle_Type and Advertising_Expenditure columns
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Pie chart for total expenditure share by vehicle type during recessions"
                )
            )

        # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
        # Hint:Use unemployment_rate,Vehicle_Type and Automobile_Sales columns
        unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
                                           x='unemployment_rate',
                                           y='Automobile_Sales',
                                           color='Vehicle_Type',
                                           labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},title='Effect of Unemployment Rate on Vehicle Type and Sales'))
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})
            ]
        
        # Yearly Statistic Report Plots 
    elif (year and stat=='Yearly Statistics'):
        yearly_data = df[df['Year']== year]
        
        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas= df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title='Yearly Automobile Sales'))

        # Total Monthly Automobile Sales
        mas=yearly_data.groupby(df['Month'])['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
            x='Month',
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))

        # Plot bar chart for average number of vehicles sold during the given year by type
        avr_vdata=yearly_data.groupby(['Year', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                             title='Average Vehicles Sold by Vehicle Type in the year {}'.format(year)))

        # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
        # Hint:Use the columns Vehicle_Type and Advertising_Expenditure
        exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
                         values='Advertising_Expenditure',
                         names='Vehicle_Type',
                         title='Total Advertisement Expenditure for Each Vehicle')
        )

        return [
                html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display':'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                ]

if __name__ == '__main__':
    app.run_server(debug=True, port=8051) # or whatever you choose